In [45]:
import urllib.request
import pandas as pd
import requests
import os
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

#url="https://github.com/cs109/2014_data/blob/master/countries.csv"
#s=requests.get(url).content
#c=pd.read_csv(s)

## План работ

1. Получить полный список торгуемых бумаг
2. Получить список бумаг, входящих в RTSI
3. Получить исторические котировки RTSI
4. Получить исторические котировки по всем торгуемым бумагам

Подготовим структуру каталогов:
* raw_data - сюда сложим все что скачаем
* output - тут буду результаты обработки

In [46]:
for d in ('./raw_data','./output','./raw_data/quotes'):
    os.makedirs(d) if (not os.path.exists(d)) else _

## 1. Получить полный список торгуемых бумаг
Да-да, самый полный. Потом выкинем что не нужно.

In [47]:
url = "http://www.moex.com/iss/apps/rms/rates.csv?"+\
"iss.only=object&limit=unlimited&sort_column=SECID&sort_order=ASC&"+\
"security_types=common_share,preferred_share,depositary_receipt,ofz_bond,cb_bond,subfederal_bond,municipal_bond,corporate_bond,exchange_bond,ifi_bond,euro_bond,public_ppif,interval_ppif,private_ppif,stock_mortgage,etf_ppif,gcc&"+\
"board_groups=stock_tplus,stock_ndm_tplus,stock_small_tplus,stock_qnv_tplus,stock_ndm_qnv_tplus,stock_d_tplus,stock_d_ndm_tplus,stock_t0,stock_ndm,stock_qnv,stock_ndm_qnv,stock_d_ndm,stock_bonds_d_main,stock_bonds_d_ndm,stock_darkpool,stock_b_spob,stock_b_psau,stock_b_auct,stock_b_psbb,stock_b_aubb,stock_repo_na,stock_repo_adr,stock_repo_shares,stock_repo_bonds,stock_repo_qnv&"+\
"index=&listname=1,2,3,_&collateral=0&currencyid=&lang=ru"
symbols = pd.read_csv(StringIO(requests.get(url).text[7:]),sep=';',index_col=False).iloc[:-3,:]
symbols.to_csv('./raw_data/symbols.csv',index=False)
print(symbols.shape)
symbols.head(3)

(2062, 12)


,SECID,DISCOUNT1,LIMIT1,DISCOUNT2,LIMIT2,DISCOUNT3,SHORTNAME,NAME,TYPENAME,ISIN,REGISTRY_CLOSE_DATE,LISTLEVEL
0,ABRD,100,2000,100.0,2001.0,100.0,АбрауДюрсо,Абрау-Дюрсо ПАО ао,Акция обыкновенная,RU000A0JS5T7,2016-11-19,3.0
1,AESL,100,2000,100.0,2001.0,100.0,АЭССЕЛЬао,ТПГ АЭССЕЛЬ ПАО ао,Акция обыкновенная,RU000A0JU8C3,NaN,3.0
2,AFKS,35,3667100,60.0,7334200.0,95.0,Система ао,"АФК ""Система"" ПАО ао",Акция обыкновенная,RU000A0DQZE3,2016-10-07,1.0


## 2. Получить список бумаг, входящих в состав RTSI

In [4]:
url = "http://moex.com/iss/statistics/engines/stock/markets/index/analytics/RTSI.csv?"+\
"iss.meta=on&iss.dp=comma&iss.df=%25Y-%25m-%25d&iss.tf=%25H%3A%25M%3A%25S&iss.dtf=%25Y.%25m.%25d%20%25H%3A%25M%3A%25S&"+\
"iss.json=extended&callback=JSON_CALLBACK&limit=100&date=2017-02-09"
rtsi_comp = pd.read_csv(StringIO(requests.get(url).text[11:].replace(",",".")),sep=';',index_col=False).iloc[:-6,:]
rtsi_comp.to_csv('./raw_data/rtsi_comp.csv',index=False)
print(rtsi_comp.shape)
rtsi_comp.head(3)

(50, 21)


,indexid,tradedate,ticker,shortnames,secids,waprice,issue_size_total,cap_total,ff_factor,w_factor,...,cap_index,weight,value,num_trades,volatility,factora,factorb,influence,determinat,currencyid
0,RTSI,2017-02-09,AFKS,AFKS,AFKS,0.41103,9.650000e+09,3.966425e+09,0.36,1.0,...,1.427913e+09,0.83,1775748.81,0.0,0.0,0.0,0.0,0.000053,0.0,USD
1,RTSI,2017-02-09,AFLT,AFLT,AFLT,3.03176,1.110616e+09,3.367122e+09,0.41,1.0,...,1.380520e+09,0.80,11349288.26,0.0,0.0,0.0,0.0,0.000335,0.0,USD
2,RTSI,2017-02-09,AGRO,AGRO,AGRO,13.99535,1.200000e+08,1.679442e+09,0.21,1.0,...,3.526828e+08,0.20,283380.93,0.0,0.0,0.0,0.0,-0.000005,0.0,USD


## 3. Получить исторические котировки RTSI

In [5]:
url = "http://export.finam.ru/RTSI_000101_170210.txt?"+\
"market=6&em=95&code=RTSI&apply=0&df=1&mf=0&yf=1991&from=01.01.1991&dt=10&mt=1&yt=2017&to=10.02.2017&"+\
"p=8&f=RTSI_910101_170210&e=.txt&cn=RTSI&dtf=1&tmf=1&MSOR=1&mstime=on&mstimever=1&sep=1&sep2=1&datf=1&at=1"
rtsi = pd.read_csv(StringIO(requests.get(url).text.replace('<','').replace('>','')),sep=',',index_col=False)
rtsi.to_csv('./raw_data/rtsi_history.csv',index=False)
rtsi.head(3)

,TICKER,PER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL
0,RTSI,D,19950901,0,100.00,100.00,100.00,100.00,2710988
1,RTSI,D,19950904,0,99.80,99.80,99.80,99.80,1000395
2,RTSI,D,19950905,0,100.42,100.42,100.42,100.42,1669902


## 4. Получить исторические котировки по всем бумагам

### 4.1 Небольшой дополнительный шаг
Как выясняется, утащить данные с finam.ru только по имени инструмента нельзя. Нужен внутренний код. Поэтому сначала утаскиваем внутренние коды, используемые на сайте.

In [43]:
#Справочник инструментов на finam.ru внезапно находится в js файле
url = "http://www.finam.ru/cache/icharts/icharts.js"
resp = requests.get(url).text.split('\r\n')
resp = [s.split(' = ') for s in resp]
dct = {}
#Там еще много всякого, поэтому придется парсить
for d in resp:
    if len(d[0])>0:
        name = d[0][4:]
        lst = d[1][1:-2].split(',')
        lst = [i.replace("\'","").replace('\"','') for i in lst]
        dct[name] = lst
#Пересобираем в DataFrame
em = pd.DataFrame({'EmId':dct['aEmitentIds'],'EmCode':dct['aEmitentCodes'],'EmMarket':dct['aEmitentMarkets']})
#Сохраняем в файл только по "МосБиржа акции"
em = em[em['EmMarket'] == '1'].sort_values(by='EmCode')[['EmCode','EmId']]
em.to_csv('./raw_data/moex.csv',index=False)
em.head(3)

,EmCode,EmId
8796,ABRD,82460
8795,AESL,181867
13458,AFKS,19715


In [49]:
#s_task = set(symbols['SECID'])
#s_task = ('RU000A0JW5J2','KBTK','RU000A0JWC25','RU000A0JNUN9','NAUK')
#df = symbols.merge(em,left_on='SECID',right_on='EmCode',how='inner')
#df.head(10)

,SECID,DISCOUNT1,LIMIT1,DISCOUNT2,LIMIT2,DISCOUNT3,SHORTNAME,NAME,TYPENAME,ISIN,REGISTRY_CLOSE_DATE,LISTLEVEL,EmCode,EmId
0,ABRD,100,2000,100.0,2001.0,100.0,АбрауДюрсо,Абрау-Дюрсо ПАО ао,Акция обыкновенная,RU000A0JS5T7,2016-11-19,3.0,ABRD,82460
1,AESL,100,2000,100.0,2001.0,100.0,АЭССЕЛЬао,ТПГ АЭССЕЛЬ ПАО ао,Акция обыкновенная,RU000A0JU8C3,NaN,3.0,AESL,181867
2,AFKS,35,3667100,60.0,7334200.0,95.0,Система ао,"АФК ""Система"" ПАО ао",Акция обыкновенная,RU000A0DQZE3,2016-10-07,1.0,AFKS,19715
3,AFLT,35,1212100,50.0,2424200.0,60.0,Аэрофлот,Аэрофлот-росс.авиалин(ПАО)ао,Акция обыкновенная,RU0009062285,2014-07-08,1.0,AFLT,29
4,AGRO,70,10638,85.0,21277.0,95.0,AGRO-гдр,ГДР ROS AGRO PLC ORD SHS,Депозитарная расписка,US7496552057,2016-09-23,1.0,AGRO,399716
5,AKRN,55,3526,85.0,7052.0,95.0,Акрон,Акрон ПАО ао,Акция обыкновенная,RU0009028674,2016-09-20,1.0,AKRN,17564
6,ALBK,100,2000,100.0,2001.0,100.0,БестЭфБ ао,Бест Эффортс Банк ПАО ао,Акция обыкновенная,RU000A0JS2M9,NaN,3.0,ALBK,82616
7,ALNU,100,2000,100.0,2001.0,100.0,АЛРОСА-Нюр,АЛРОСА-Нюрба ПАО ао,Акция обыкновенная,RU000A0JP468,2016-12-23,3.0,ALNU,81882
8,ALRS,23,2167200,38.0,4334400.0,50.0,АЛРОСА ао,АЛРОСА ПАО ао,Акция обыкновенная,RU0007252813,2016-07-19,1.0,ALRS,81820
9,AMEZ,100,2000,100.0,2001.0,100.0,АшинскийМЗ,Ашинский метзавод ПАО ао,Акция обыкновенная,RU000A0B88G6,NaN,2.0,AMEZ,20702


In [55]:
for ix,r in em.iterrows():
    s = r['EmCode']
    i = r['EmId']
    print ("Downloading:",s,i)
    try:
        urllib.request.urlretrieve(
            "http://export.finam.ru/"+s+".csv?d=d&m=1&em="+str(i)+"&"+\
            "df=1&mf=1&yf=2000&dt=1&mt=2&yt=2017&"+\
            "p=8&f="+s+"&e=.csv&cn="+s+"&"+\
            "dtf=1&tmf=1&MSOR=0&sep=1&sep2=1&datf=4&at=1", './raw_data/quotes/'+s+".csv")
    except:
        print(s,"download failed")

Downloading: ABRD 82460
Downloading: AESL 181867
Downloading: AFKS 19715
Downloading: AFLT 29
Downloading: AGRO 399716
Downloading: AKRN 17564
Downloading: ALBK 82616
Downloading: ALNU 81882
Downloading: ALRS 81820
Downloading: AMEZ 20702
Downloading: APTK 13855
Downloading: AQUA 35238
Downloading: ARMD 19676
Downloading: ARSA 19915
Downloading: ASSB 16452
Downloading: AVAN 82843
Downloading: AVAZ 39
Downloading: AVAZP 40
Downloading: BANE 81757
Downloading: BANEP 81758
Downloading: BGDE 175840
Downloading: BISV 35242
Downloading: BISVP 35243
Downloading: BLNG 21078
Downloading: BRZL 81901
Downloading: BSPB 20066
Downloading: CBOM 420694
Downloading: CHEP 20999
Downloading: CHGZ 81933
Downloading: CHKZ 21000
Downloading: CHMF 16136
Downloading: CHMK 21001
Downloading: CHZN 19960
Downloading: CLSB 16712
Downloading: CLSBP 16713
Downloading: CNTL 21002
Downloading: CNTLP 81575
Downloading: DASB 16825
Downloading: DGBZ 17919
Downloading: DIOD 35363
Downloading: DIXY 18564
Downloading: DSK

In [40]:
#Справочник инструментов на finam.ru внезапно находится в js файле
url = "http://www.finam.ru/cache/icharts/icharts.js"
resp = requests.get(url).text.split('\r\n')
resp = [s.split(' = ') for s in resp]
dct = {}
#Там еще много всякого, поэтому придется парсить
for d in resp:
    if len(d[0])>0:
        name = d[0][4:]
        lst = d[1][1:-2].split(',')
        lst = [i.replace("\'","").replace('\"','') for i in lst]
        dct[name] = lst
#Пересобираем в DataFrame
df = pd.DataFrame({'EmId':dct['aEmitentIds'],'EmCode':dct['aEmitentCodes'],'EmMarket':dct['aEmitentMarkets']})
#Сохраняем в файл только по "МосБиржа акции"
df[df['EmMarket'] == '1'].sort_values(by='EmCode')[['EmCode','EmId']].to_csv('./raw_data/moex.csv',index=False)